# Import Formated Cash Flow Statement from DAI Data Platform

## 1. Library & Configuration

In [1]:
import dai
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

## 2. Data Collection

In [3]:
sd = '2023-01-01'
ed = '2026-01-01'

In [4]:
company_code = '600010.SH'
report_date = '2024-12-31'
change_type = '0'

In [5]:
sql = f"""

SELECT *
FROM cn_stock_financial_cashflow_general_pit
WHERE 1=1
AND change_type = {change_type}
AND instrument = '{company_code}'
AND report_date = '{report_date}'

"""

In [6]:
df = dai.query(sql, filters = {'date':[sd, ed]}).df().squeeze()
df.dropna()

date                                            2025-04-19 00:00:00
instrument                                                600010.SH
report_date                                     2024-12-31 00:00:00
fs_quarter_index                                                  4
change_type                                                       0
cash_paid_for_debt_repayment                      23,619,062,167.61
loss_from_fair_value_chg                              -3,852,550.00
others_in_cashflow_sheet                              22,292,990.78
cash_paid_for_dividends_profits_interests          1,451,477,907.54
cash_received_from_bond_issue                        499,500,000.00
cash_received_from_borrowings                     26,847,661,618.96
return_on_investment                                  22,843,159.87
capital_contributions_received                        66,987,530.09
depreciation_of_fa_oga_pba                         4,668,775,504.83
loss_from_scraping_of_fixed_assets              

## 3. Cash Flow Statement Items Classification

In [7]:
item_dict_cf = {

# =========================================================
# 一、经营活动产生的现金流量
# =========================================================
"经营活动产生的现金流量": {

    # ----------------------------
    # 1. 直接法
    # ----------------------------
    "直接法": {

        # 经营流入
        "经营活动现金流入": {
            "销售商品、提供劳务收到的现金": df.loc["cash_received_from_sales_and_services"],
            "客户存款和同业存放款项净增加额": df.loc["netinc_in_deposits"],
            "向中央银行借款净增加额": df.loc["netinc_in_borrowings_from_central_bank"],
            "向其他金融机构拆入资金净增加额": df.loc["netinc_in_loans_from_other_fin_institutions"],
            "收到原保险合同保费取得的现金": df.loc["cash_received_from_premiums"],
            "收到再保业务现金净额": df.loc["net_cash_received_from_reinsurance"],
            "保户储金及投资款净增加额": df.loc["netinc_in_insurance_deposits_and_invest"],
            "处置交易性金融资产净增加额": df.loc["netinc_in_disposal_fin_assets"],
            "收取利息、手续费及佣金的现金": df.loc["cash_received_from_interests_fess_and_commissions"],
            "拆入资金净增加额": df.loc["netinc_in_loans_from_banks_and_fin_institutions"],
            "回购业务资金净增加额": df.loc["netinc_in_repurchase_transactions"],
            "收到的税费返还": df.loc["taxes_and_levies_rebates"],
            "收到其他与经营活动有关的现金": df.loc["cash_received_from_other_operating"],
        },

        "经营活动现金流入小计": {
            "合计": df.loc["subtotal_cifoa"]
        },

        # 经营流出
        "经营活动现金流出": {
            "购买商品、接受劳务支付的现金": df.loc["cash_paid_for_goods_and_services"],
            "客户贷款及垫款净增加额": df.loc["netinc_in_loans_and_advances"],
            "存放中央银行和同业款项净增加额": df.loc["netinc_deposits_central_bank_interbank"],
            "支付原保险合同赔付款项的现金": df.loc["cash_paid_for_claims"],
            "支付利息、手续费及佣金的现金": df.loc["cash_paid_for_interests_fees_and_commissions"],
            "支付保单红利的现金": df.loc["cash_paid_for_policy_dividends"],
            "支付给职工以及为职工支付的现金": df.loc["cash_paid_for_employees"],
            "支付的各项税费": df.loc["cash_paid_for_taxes_and_levies"],
            "支付其他与经营活动有关的现金": df.loc["other_cofoa"],
        },

        "经营活动现金流出小计": {
            "合计": df.loc["subtotal_cofoa"]
        },

        "经营活动现金净流入": {
            "合计": df.loc["net_cffoa"]
        },
    },

    # ----------------------------
    # 2. 间接法
    # ----------------------------
    "间接法": {

        "经营净利润及调整项": {
            "净利润": df.loc["net_profit_in_cashflow_sheet"],
            "信用减值损失": df.loc["credit_impairment_loss_in_cashflow_sheet"],
            "资产减值准备": df.loc["asset_impairment_reserve"],
            "固定资产折旧等": df.loc["depreciation_of_fa_oga_pba"],
            "无形资产摊销": df.loc["amorization_of_intangible_assets"],
            "长期待摊费用摊销": df.loc["amortization_of_longterm_deferred_expenses"],
            "处置固定资产等损失": df.loc["loss_from_disposal_of_fa_ia_lta"],
            "固定资产报废损失": df.loc["loss_from_scraping_of_fixed_assets"],
            "公允价值变动损失": df.loc["loss_from_fair_value_chg"],
            "财务费用": df.loc["finance_expenses_in_cashflow_sheet"],
            "投资损失": df.loc["invest_loss"],
            "递延所得税资产减少": df.loc["decrease_in_deferred_tax_assets"],
            "递延所得税负债增加": df.loc["increase_in_deferred_tax_liabilities"],
            "存货的减少": df.loc["decrease_in_inventories"],
            "经营性应收项目的减少": df.loc["decrease_in_operating_receivables"],
            "经营性应付项目的增加": df.loc["increase_in_operating_payables"],
            "其他": df.loc["others_in_cashflow_sheet"],
        },

        "经营活动现金净流入": {
            "合计": df.loc["net_cffoa_indirect"]
        }
    }
},

# =========================================================
# 二、投资活动产生的现金流量
# =========================================================
"投资活动产生的现金流量": {
    # 无表示不分直接间接
    "无": {

        "投资活动现金流入": {
            "收回投资收到的现金": df.loc["cash_received_from_disposal_investments"],
            "取得投资收益收到的现金": df.loc["return_on_investment"],
            "处置固定资产等收回的现金净额": df.loc["net_cash_received_from_disposal_filt_assets"],
            "处置子公司等收到的现金净额": df.loc["net_cash_received_from_disposal_subsidiaries"],
            "收到其他与投资活动有关的现金": df.loc["cash_received_from_other_investing"],
        },

        "投资活动现金流入小计": {
            "合计": df.loc["subtotal_cifia"]
        },

        "投资活动现金流出": {
            "购建固定资产等支付的现金": df.loc["cash_paid_for_filt_assets"],
            "投资支付的现金": df.loc["cash_paid_for_investments"],
            "质押贷款净增加额": df.loc["netinc_in_pledge_loans"],
            "取得子公司等支付的现金净额": df.loc["cash_paid_by_acquiring_subsidiaries"],
            "支付其他与投资活动有关的现金": df.loc["cash_paid_for_other_investing"],
        },

        "投资活动现金流出小计": {
            "合计": df.loc["subtotal_of_cofia"]
        },

        "投资活动现金净流入": {
            "合计": df.loc["net_cffia"]
        },
    }
},

# =========================================================
# 三、筹资活动产生的现金流量
# =========================================================
"筹资活动产生的现金流量": {
    # 无表示不分直接间接
    "无": {

        "筹资活动现金流入": {
            "吸收投资收到的现金": df.loc["capital_contributions_received"],
            "其中：子公司吸收少数股东投资收到的现金": df.loc["cash_received_by_subsidiaries_from_minority"],
            "取得借款收到的现金": df.loc["cash_received_from_borrowings"],
            "发行债券收到的现金": df.loc["cash_received_from_bond_issue"],
            "收到其他与筹资活动有关的现金": df.loc["cash_received_from_other_financing"],
        },

        "筹资活动现金流入小计": {
            "合计": df.loc["subtotal_ciffa"]
        },

        "筹资活动现金流出": {
            "偿还债务支付的现金": df.loc["cash_paid_for_debt_repayment"],
            "分配股利、利润或偿付利息支付的现金": df.loc["cash_paid_for_dividends_profits_interests"],
            "其中：子公司支付给少数股东的股利、利润": df.loc["cash_paid_by_subsidiaries_to_minority"],
            "支付其他与筹资活动有关的现金": df.loc["cash_paid_for_other_financing"],
        },

        "筹资活动现金流出小计": {
            "合计": df.loc["subtotal_of_coffa"]
        },

        "筹资活动现金净流入": {
            "合计": df.loc["net_cfffa"]
        },
    }
},

# =========================================================
# 四、汇率变动影响
# =========================================================
"汇率变动对现金及现金等价物的影响": {
    # 无表示不分直接间接
    "无": {
        "汇率变动对现金及现金等价物的影响": {
            "合计": df.loc["effect_of_exchange_chg_on_cce"]
        }
    }
},

# =========================================================
# 五、现金流量表汇总
# =========================================================
"现金流量表汇总": {
    # 无表示不分直接间接
    "无": {
        "现金及现金等价物净增加额": {
            "合计": df.loc["netinc_in_cce"]
        },
        "期初现金及现金等价物余额": {
            "合计": df.loc["cce_beginning"]
        },
        "期末现金及现金等价物余额": {
            "合计": df.loc["cce_ending"]
        }
    }
},

# =========================================================
# 六、补充资料
# =========================================================
"补充资料": {
    # 无表示不分直接间接
    "无": {
        "现金与等价物构成": {
            "现金的期初余额": df.loc["cash_balance_beginning"],
            "现金的期末余额": df.loc["cash_balance_ending"],
            "现金等价物的期末余额": df.loc["cce_balance_ending"],
        }
    }
}

}


## 4. Cash Flow Statement Generation

In [8]:
def get_df_cf_from_item_dict(item_dict_cf, df):

    def to_num0(x):
        if x is None:
            return 0.0
        if isinstance(x, pd.Series):
            x = x.iloc[0] if x.size > 0 else 0.0
        if isinstance(x, pd.DataFrame):
            x = x.iloc[0, 0] if not x.empty else 0.0
        try:
            if pd.isna(x):
                return 0.0
        except Exception:
            pass
        if isinstance(x, str):
            s = x.strip().replace(",", "")
            if s == "" or s.lower() == "nan":
                return 0.0
            try:
                return float(s)
            except Exception:
                return 0.0
        try:
            v = float(x)
            return 0.0 if (isinstance(v, float) and np.isnan(v)) else v
        except Exception:
            return 0.0

    rows, vals = [], []
    seq_map = {}
    _seq = 0

    def walk(node, path):
        nonlocal _seq
        if isinstance(node, dict):
            for k, v in node.items():
                walk(v, path + [k])
        else:
            p = path[:]
            while len(p) < 4:
                p.append("")
            if len(p) > 4:
                head = p[:3]
                tail = " / ".join(map(str, p[3:]))
                p = head + [tail]
            key = tuple(p)
            rows.append(key)
            vals.append(to_num0(node))
            seq_map[key] = _seq
            _seq += 1

    walk(item_dict_cf, [])

    base = pd.DataFrame(
        {"原表数值": vals},
        index=pd.MultiIndex.from_tuples(
            rows, names=["一级分类", "二级分类", "三级分类", "项目"]
        )
    )

    def safe_scalar_loc(key, default=""):
        try:
            x = df.loc[key]
        except Exception:
            return default
        if isinstance(x, pd.Series):
            x = x.iloc[0] if x.size > 0 else default
        if isinstance(x, pd.DataFrame):
            x = x.iloc[0, 0] if not x.empty else default
        try:
            if pd.isna(x):
                return default
        except Exception:
            pass
        return x

    def to_date_str(x):
        try:
            return str(pd.to_datetime(x).date())
        except Exception:
            return str(x) if x is not None else ""

    instrument = str(safe_scalar_loc("instrument", ""))
    report_date = to_date_str(safe_scalar_loc("report_date", ""))
    announce_date = to_date_str(safe_scalar_loc("date", ""))

    header_rows = [
        ("基本信息", "无", "基本信息", "报表类型", "现金流量表"),
        ("基本信息", "无", "基本信息", "公司代码", instrument),
        ("基本信息", "无", "基本信息", "报表日期", report_date),
        ("基本信息", "无", "基本信息", "公告日期", announce_date),
    ]
    header_index = pd.MultiIndex.from_tuples(
        [(a, b, c, d) for a, b, c, d, _ in header_rows],
        names=base.index.names
    )
    header_df = pd.DataFrame(
        {"原表数值": [v for *_, v in header_rows]},
        index=header_index
    )
    for i, (a, b, c, d, _) in enumerate(header_rows):
        seq_map[(a, b, c, d)] = -10_000 + i

    base = pd.concat([header_df, base], axis=0)

    def ensure_row(l1, l2, l3, item="合计", init_val=0.0):
        idx = (l1, l2, l3, item)
        if idx not in base.index:
            base.loc[idx, "原表数值"] = float(init_val)
            seq_map[idx] = 10**12 if item == "合计" else 10**11

    ensure_row("经营活动产生的现金流量", "直接法", "经营活动现金流入小计")
    ensure_row("经营活动产生的现金流量", "直接法", "经营活动现金流出小计")
    ensure_row("经营活动产生的现金流量", "直接法", "经营活动现金净流入")

    ensure_row("经营活动产生的现金流量", "间接法", "经营活动现金净流入")

    ensure_row("投资活动产生的现金流量", "无", "投资活动现金流入小计")
    ensure_row("投资活动产生的现金流量", "无", "投资活动现金流出小计")
    ensure_row("投资活动产生的现金流量", "无", "投资活动现金净流入")

    ensure_row("筹资活动产生的现金流量", "无", "筹资活动现金流入小计")
    ensure_row("筹资活动产生的现金流量", "无", "筹资活动现金流出小计")
    ensure_row("筹资活动产生的现金流量", "无", "筹资活动现金净流入")

    ensure_row("汇率变动对现金及现金等价物的影响", "无", "汇率变动对现金及现金等价物的影响")

    ensure_row("现金流量表汇总", "无", "现金及现金等价物净增加额")
    ensure_row("现金流量表汇总", "无", "期初现金及现金等价物余额")
    ensure_row("现金流量表汇总", "无", "期末现金及现金等价物余额")

    def sum_block(l1, l2, l3):
        mask = (
            (base.index.get_level_values(0) == l1) &
            (base.index.get_level_values(1) == l2) &
            (base.index.get_level_values(2) == l3) &
            (base.index.get_level_values(3) != "合计")
        )
        part = base.loc[mask, "原表数值"]
        if part.empty:
            return 0.0

        items = part.index.get_level_values(3).astype(str)

        keep = ~items.str.contains("差额", na=False)

        keep &= ~items.str.startswith("其中：", na=False)

        part = part[keep]
        return float(part.map(to_num0).sum())

    def get_val_num(l1, l2, l3, item="合计"):
        idx = (l1, l2, l3, item)
        if idx not in base.index:
            return 0.0
        return to_num0(base.loc[idx, "原表数值"])

    base["计算合计数值"] = np.nan

    def set_calc(l1, l2, l3, val):
        base.loc[(l1, l2, l3, "合计"), "计算合计数值"] = float(val)

    l1 = "经营活动产生的现金流量"
    cifoa = sum_block(l1, "直接法", "经营活动现金流入")
    cofoa = sum_block(l1, "直接法", "经营活动现金流出")
    set_calc(l1, "直接法", "经营活动现金流入小计", cifoa)
    set_calc(l1, "直接法", "经营活动现金流出小计", cofoa)
    set_calc(l1, "直接法", "经营活动现金净流入", cifoa - cofoa)

    indirect_sum = sum_block("经营活动产生的现金流量", "间接法", "经营净利润及调整项")
    set_calc("经营活动产生的现金流量", "间接法", "经营活动现金净流入", indirect_sum)

    l1 = "投资活动产生的现金流量"
    cifia = sum_block(l1, "无", "投资活动现金流入")
    cofia = sum_block(l1, "无", "投资活动现金流出")
    set_calc(l1, "无", "投资活动现金流入小计", cifia)
    set_calc(l1, "无", "投资活动现金流出小计", cofia)
    set_calc(l1, "无", "投资活动现金净流入", cifia - cofia)

    l1 = "筹资活动产生的现金流量"
    ciffa = sum_block(l1, "无", "筹资活动现金流入")
    coffa = sum_block(l1, "无", "筹资活动现金流出")
    set_calc(l1, "无", "筹资活动现金流入小计", ciffa)
    set_calc(l1, "无", "筹资活动现金流出小计", coffa)
    set_calc(l1, "无", "筹资活动现金净流入", ciffa - coffa)

    fx = get_val_num("汇率变动对现金及现金等价物的影响", "无",
                     "汇率变动对现金及现金等价物的影响", "合计")
    set_calc("汇率变动对现金及现金等价物的影响", "无",
             "汇率变动对现金及现金等价物的影响", fx)

    netinc = (cifoa - cofoa) + (cifia - cofia) + (ciffa - coffa) + fx
    set_calc("现金流量表汇总", "无", "现金及现金等价物净增加额", netinc)

    beg = get_val_num("现金流量表汇总", "无", "期初现金及现金等价物余额", "合计")
    set_calc("现金流量表汇总", "无", "期末现金及现金等价物余额", beg + netinc)

    def is_header_row(l1):
        return l1 == "基本信息"

    def is_outflow_l3(l3):
        return "现金流出" in str(l3)

    add_vals, sub_vals = [], []
    for (l1, l2, l3, item), r in base.iterrows():
        if is_header_row(l1):
            add_vals.append(r["原表数值"])
            sub_vals.append(np.nan)
            continue
        v = to_num0(r["原表数值"])
        if is_outflow_l3(l3):
            add_vals.append(np.nan)
            sub_vals.append(v)
        else:
            add_vals.append(v)
            sub_vals.append(np.nan)

    out = base.copy()
    out["加法数值"] = add_vals
    out["减法数值"] = sub_vals
    out = out[["加法数值", "减法数值", "计算合计数值"]]

    blocks = [
        ("基本信息", "无", "基本信息"),

        ("经营活动产生的现金流量", "直接法", "经营活动现金流入"),
        ("经营活动产生的现金流量", "直接法", "经营活动现金流入小计"),
        ("经营活动产生的现金流量", "直接法", "经营活动现金流出"),
        ("经营活动产生的现金流量", "直接法", "经营活动现金流出小计"),
        ("经营活动产生的现金流量", "直接法", "经营活动现金净流入"),

        ("经营活动产生的现金流量", "间接法", "经营净利润及调整项"),
        ("经营活动产生的现金流量", "间接法", "经营活动现金净流入"),

        ("投资活动产生的现金流量", "无", "投资活动现金流入"),
        ("投资活动产生的现金流量", "无", "投资活动现金流入小计"),
        ("投资活动产生的现金流量", "无", "投资活动现金流出"),
        ("投资活动产生的现金流量", "无", "投资活动现金流出小计"),
        ("投资活动产生的现金流量", "无", "投资活动现金净流入"),

        ("筹资活动产生的现金流量", "无", "筹资活动现金流入"),
        ("筹资活动产生的现金流量", "无", "筹资活动现金流入小计"),
        ("筹资活动产生的现金流量", "无", "筹资活动现金流出"),
        ("筹资活动产生的现金流量", "无", "筹资活动现金流出小计"),
        ("筹资活动产生的现金流量", "无", "筹资活动现金净流入"),

        ("汇率变动对现金及现金等价物的影响", "无", "汇率变动对现金及现金等价物的影响"),

        ("现金流量表汇总", "无", "现金及现金等价物净增加额"),
        ("现金流量表汇总", "无", "期初现金及现金等价物余额"),
        ("现金流量表汇总", "无", "期末现金及现金等价物余额"),

        ("补充资料", "无", "现金与等价物构成"),
        ("补充资料", "无", "非现金项目"),
    ]
    block_order = {b: i for i, b in enumerate(blocks)}

    idx_df = out.index.to_frame(index=False)

    def item_rank(item, key):
        if item == "合计":
            return 10**13
        return seq_map.get(key, 10**11)

    idx_df["_bord"] = idx_df.apply(
        lambda r: block_order.get((r["一级分类"], r["二级分类"], r["三级分类"]), 10**9),
        axis=1
    )
    idx_df["_irank"] = idx_df.apply(
        lambda r: item_rank(r["项目"], (r["一级分类"], r["二级分类"], r["三级分类"], r["项目"])),
        axis=1
    )

    idx_df = idx_df.sort_values(["_bord", "_irank"]).drop(columns=["_bord", "_irank"])
    out = out.loc[pd.MultiIndex.from_frame(idx_df)]

    return out


In [10]:
df_cf = get_df_cf_from_item_dict(item_dict_cf, df)
df_cf

加法数值  \
一级分类             二级分类 三级分类             项目                                      
基本信息             无    基本信息             报表类型                            现金流量表   
                                       公司代码                        600010.SH   
                                       报表日期                       2024-12-31   
                                       公告日期                       2025-04-19   
经营活动产生的现金流量      直接法  经营活动现金流入         销售商品、提供劳务收到的现金      72,196,234,025.71   
                                       客户存款和同业存放款项净增加额                  0.00   
                                       向中央银行借款净增加额                      0.00   
                                       向其他金融机构拆入资金净增加额                  0.00   
                                       收到原保险合同保费取得的现金                   0.00   
                                       收到再保业务现金净额                       0.00   
                                       保户储金及投资款净增加额                     0.00   
                                       处置交易性金融资产净增加额                    0.00   
                                       收取利息、手续费及佣金的现金                   0.00   
                                       拆入资金净增加额                         0.00   
                                       回购业务资金净增加额                       0.00   
                                       收到的税费返还                128,604,050.06   
                                       收到其他与经营活动有关的现金         760,690,609.96   
                      经营活动现金流入小计       合计                  73,085,528,685.73   
                      经营活动现金流出         购买商品、接受劳务支付的现金                    NaN   
                                       客户贷款及垫款净增加额                       NaN   
                                       存放中央银行和同业款项净增加额                   NaN   
                                       支付原保险合同赔付款项的现金                    NaN   
                                       支付利息、手续费及佣金的现金                    NaN   
                                       支付保单红利的现金                         NaN   
                                       支付给职工以及为职工支付的现金                   NaN   
                                       支付的各项税费                           NaN   
                                       支付其他与经营活动有关的现金                    NaN   
                      经营活动现金流出小计       合计                                NaN   
                      经营活动现金净流入        合计                   2,377,288,640.25   
                 间接法  经营净利润及调整项        净利润                     20,748,912.46   
                                       信用减值损失                           0.00   
                                       资产减值准备                 391,061,292.33   
                                       固定资产折旧等              4,668,775,504.83   
                                       无形资产摊销                 136,527,446.65   
                                       长期待摊费用摊销                38,197,265.17   
                                       处置固定资产等损失              -45,148,640.57   
                                       固定资产报废损失                36,926,967.22   
                                       公允价值变动损失                -3,852,550.00   
                                       财务费用                 1,977,859,745.50   
                                       投资损失                  -344,904,075.88   
                                       递延所得税资产减少               15,855,124.71   
                                       递延所得税负债增加               -4,060,944.30   
                                       存货的减少                  530,954,856.50   
                                       经营性应收项目的减少          -1,657,518,057.56   
                                       经营性应付项目的增加          -3,411,200,037.89   
                                       其他                      22,292,990.78   
                      经营活动现金净流入        合计                   2,377,288,640.25   
投资活动产生的现金流量      无    投资活动现金流入         收回投资收到的现金                        0.00   
                                       取得投资收益收到的现金             22,843,15

In [ ]:
df_cf.to_csv(f"Cash Flow Statement {company_code} {report_date}.csv")